# Basic CSP Startup Script

Before starting the notebook, please run the job `deploy-sut-integration` of the `integration` stage in the latest gitlab pipeline: https://gitlab.com/ska-telescope/ska-mid-itf/-/pipelines?page=1&scope=all&ref=main. 

<div class="alert alert-block alert-danger">
<b>WARNING</b>The CBF configuration scripts' output is extremely verbose and will cause the CI job outputs to be too large to download in a single CI job. In future this might be improved.</div>

In [7]:
import os
import datetime, json
from tango import DeviceProxy, Database

# Take the namespace name from the deployment job
KUBE_NAMESPACE = "ci-ska-mid-itf-at-1702-add-csplmc-tests"
# KUBE_NAMESPACE = "integration"
CLUSTER_DOMAIN = "miditf.internal.skao.int"
# set the name of the databaseds service
DATABASEDS_NAME = "tango-databaseds"

# finally set the TANGO_HOST
os.environ[
    "TANGO_HOST"
] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"
TIMEOUT=60

List all devices available:

In [ ]:
database = Database()
instance_list = database.get_device_exported("*")
for instance in instance_list.value_string:
    dev = DeviceProxy(instance)
    print(instance)
    print("*****")
    print(dev.info())

## Setup Device connection and Timeouts
Set up DeviceProxy, check AdminMode and if not 0, set to 0 (ONLINE)

In [23]:
CSP = DeviceProxy("mid-csp/control/0")

In [24]:
CSP.adminmode # This is likely currently OFFLINE

<adminMode.ONLINE: 0>

In [25]:
CSP.State() # This should now be DISABLE

tango._tango.DevState.ON

Set Adminmode to ONLINE and check state

In [26]:
CSP.adminmode=0
CSP.adminmode

<adminMode.ONLINE: 0>

In [27]:
CSP.State() # This should now be OFF

tango._tango.DevState.ON

Check CBF SimulationMode (this should be FALSE for real hardware control)

In [28]:
CSP.cbfSimulationMode # Should be True if hardware is not currently under CSP.LMC control

False

In [29]:
CSP.cbfSimulationMode=False
CSP.cbfSimulationMode

False

Set Timeout to 60 seconds as the ON command is a long-running command and check value

In [30]:
CSP.commandTimeout=TIMEOUT
CSP.commandTimeout

60

## Check all relevant device states
Now check if all the devices are switched on as they should be.
Use the [Taranta Dashboard](https://k8s.miditf.internal.skao.int/integration/taranta/dashboard?id=6543b36cf6d53e0019b95cf1&mode=run) to monitor top-level devices!

## Send command sequence
### Send the ON command. 
NOTE: an empty list sends the ON command to ALL the subsystems, specific subsystems are turned on if specified in a list of subsystem FQDNs

In [ ]:
CSP.on([])

In [ ]:
CSP.State() # this should now be ON

### Control the CSP Subarray
Set up a Tango DeviceProxy to the CSP Subarray device


In [4]:
CSPSubarray=DeviceProxy('mid-csp/subarray/01')

### Send AssignResource

In [31]:
resources=json.dumps({
    "subarray_id": 1,
    "dish":{
        "receptor_ids":["SKA001"]
    }
})

CSPSubarray.AssignResources(resources)

DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: AssignResources command not permitted in observation state FAULT
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/subarray/subarray_device.py", line 574, in is_AssignResources_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: AssignResources command not permitted in observation state FAULT

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device mid-csp/subarray/01, command AssignResources
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at 

### Send ConfigureScan command
NOTE: get the IP address of the interface of the SDP Surrogate pod and modify the JSON for the configure scan with it, before you run this command!!!

Send this directly to the CBF Subarray device - refer to https://k8s.miditf.internal.skao.int/ci-ska-mid-itf-at-1702-add-csplmc-tests/taranta/dashboard?id=655c5c2d72024f001940aca8 for the correct command (open the button config in Taranta).


#### Copy this JSON and modify 
Uncomment the first line after getting it from the kubectl command output and modifying SDP_HOST_IP_ADDRESS

In [27]:
SDP_HOST_IP_ADDRESS="10.165.20.30" # this is the IP address of the receiver interface
configuredata={ "interface": "https://schema.skao.int/ska-csp-configure/2.3", "subarray": { "subarray_name": "Single receptor" }, "common": { "config_id": "1 receptor, band 1, 1 FSP, no options", "frequency_band": "1", "subarray_id": 0 }, "cbf": { "delay_model_subscription_point": "ska_mid/tm_leaf_node/csp_subarray_01/delayModel", "rfi_flagging_mask": {}, "fsp": [ { "fsp_id": 1, "function_mode": "CORR", "receptors": ["SKA001"], "frequency_slice_id": 1, "zoom_factor": 1, "zoom_window_tuning": 450000, "integration_factor": 10, "channel_offset": 14880, "output_link_map": [ [0, 4], [744, 8], [1488, 12], [2232, 16], [2976, 20], [3720, 24], [4464, 28], [5206, 32], [5952, 36], [6696, 40], [7440, 44], [8184, 48], [8928, 52], [9672, 56], [10416, 60], [11160, 64], [11904, 68], [12648, 72], [13392, 76], [14136, 80] ], "output_host": [[0, "10.165.20.31"]], "output_port": [[0, 14000, 1]] } ] } }
configuredata["cbf"]["fsp"][0]["output_host"] = SDP_HOST_IP_ADDRESS

In [29]:
CBFSubarray = DeviceProxy('mid_csp_cbf/sub_elt/subarray_01')

In [30]:
CBFSubarray.ConfigureScan(json.dumps(configuredata))

DevFailed: DevFailed[
DevError[
    desc = Receptor SKA001 does not belong to subarray 1.
  origin = ConfigureScan execution
  reason = Command failed
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device mid_csp_cbf/sub_elt/subarray_01, command ConfigureScan
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/src/cppTango/cppapi/client/devapi_base.cpp:1338)
  reason = API_CommandFailed
severity = ERR]
]

## Setup BITE data stream
Copy the output of the next print line and paste into your shell.

In [34]:
print(f"kubectl -n {KUBE_NAMESPACE} exec ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=1")

kubectl -n ci-ska-mid-itf-at-1702-add-csplmc-tests exec ec-bite -- python3 midcbf_bite.py --talon-bite-lstv-replay --boards=1


### Upload the Delaymodel


In [43]:
# Generate the Delaymodel and check if it was was correctly sent:
sub = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray_01")
current_time = float(datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc).timestamp())
dm={"interface": "https://schema.skao.int/ska-csp-delaymodel/2.2", "epoch": current_time, "validity_period": 400.0, "delay_details": [{"receptor": "SKA001", "poly_info": [{"polarization": "X", "coeffs": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}, {"polarization": "Y", "coeffs": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}]}]}
sub.delayModel=json.dumps(dm)
assert sub.delayModel==json.dumps(dm), f"Expected {dm}, got\n{sub.delayModel}"

## Scan
By now, we are supposed to look at the output of `tcpdump -i net1` in the SDP Surrogate pod and see data!

## End Scan (CSP Subarray)
If this fails because things were in READY, that means you didn't need to run it now - DON'T PANIC!

Run this if device is in 

In [ ]:
CSPSubarray.EndScan()

## Go To Idle (CSP Subarray)
CSPSubarray.GoToIdle()

In [32]:
CSPSubarray.obsState

<obsState.FAULT: 9>

## Release Resources (CSP Subarray)
### Run either ReleaseAllResources, or ReleaseResources(<samejsonfilefortheothercommand>)

In [33]:
CSPSubarray.ReleaseAllResources()

DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: ReleaseAllResources command not permitted in observation state FAULT
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/subarray/subarray_device.py", line 661, in is_ReleaseAllResources_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: ReleaseAllResources command not permitted in observation state FAULT

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device mid-csp/subarray/01, command ReleaseAllResources
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::

## Final Teardown
NOTE: This will off the CSP and CBF and should only be done if you don't want to use the system again.

In [ ]:
CSP.off([]) # Check with make itf-cbf-talonlru-status - lru should be off now
CSP.cbfSimulationMode=True
CSP.commandTimeout=3
CSP.adminmode=1